In [ ]:
# Données utilisées depuis le Bucket S3:

# unix_arborescence_competences_v459_utf8.csv
# unix_coherence_item_v459_utf8.csv
# unix_referentiel_code_rome_v459_utf8.csv
# unix_referentiel_competence_v459_utf8.csv

# 1:
# unix_arborescence_competences_v459_utf8.csv
# data_arbo.code_ogr_competence
# unix_coherence_item_v459_utf8.csv
# data_cohe.code_ogr 

# 2:
# df_merge.code_rome
# unix_referentiel_code_rome_v459_utf8.csv
# data_ref_rome.code_rome

# 3:
# df_merge.['libelle_competence', 'code_ogr_competence']
# unix_referentiel_competence_v459_utf8.csv
# data_ref_comp.['libelle_competence', 'code_ogr']

In [2]:
# Install specific libraries for Amazon S3 connection:
%pip install boto3
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import io
import os
from dotenv import load_dotenv
import boto3

In [2]:
# Modification des paramèters d'affichage de Pandas:
pd.set_option('display.max_row', 111)
pd.set_option('display.max_column', 111)

In [3]:
# Load environment variables with specific location depend on EDI (PyCharm/VSCode)
if not load_dotenv('../settings/.env'):
    print("Load env from alternative from path")
    load_dotenv('script/settings/.env')

# Check settings (for debug session only)
if __debug__:
    print('Debug ON')
    print("Environment data:", 
        "\nS3_ENDPOINT_URL:", os.getenv("S3_ENDPOINT_URL"),
        "\nS3_ACCESS_KEY_ID (len):", len(os.getenv("S3_ACCESS_KEY_ID")), "bytes",
        "\nS3_SECRET_ACCESS_KEY (len):", len(os.getenv("S3_SECRET_ACCESS_KEY")), "bytes",
        "\nS3_REGION:", os.getenv("S3_REGION")
        )

Debug ON
Environment data: 
S3_ENDPOINT_URL: https://bhckzdwrhhfaxbidmwpm.supabase.co/storage/v1/s3 
S3_ACCESS_KEY_ID (len): 32 bytes 
S3_SECRET_ACCESS_KEY (len): 64 bytes 
S3_REGION: eu-west-3


In [4]:
# Init S3 client 
try:
    s3_client = boto3.client(
        service_name='s3',
        region_name=os.getenv("S3_REGION"),
        endpoint_url=os.getenv("S3_ENDPOINT_URL"),
        aws_access_key_id=os.getenv("S3_ACCESS_KEY_ID"), 
        aws_secret_access_key=os.getenv("S3_SECRET_ACCESS_KEY")
    ) 
except Exception as ex:
    print(ex)

# Define function to read ROME CSV files
def read_rome_from_s3(s3_file_key):
    bucket_name = 'raw-data'
    s3_rome_path = 'CodeROME/RefRomeCsv/'

    try:
        obj = s3_client.get_object(Bucket= bucket_name, Key= s3_rome_path + s3_file_key)
        df = pd.read_csv(io.BytesIO(obj['Body'].read()))
        return df
    except Exception as ex:
        print("Erreur de lecture du fichier depuis le buecket S3:")
        print("->", ex)
    
    return None

# S3 CSV reading test
df = read_rome_from_s3( 'unix_arborescence_centre_interet_v459_utf8.csv')
if df is not None:
    print("Lecture du fichier réussi:")
    print(f"-> CSV: Nb de lignes: {df.shape[0]}, Nb de colonnes: {df.shape[1]}")

Lecture du fichier réussi:
-> CSV: Nb de lignes: 4448, Nb de colonnes: 6


In [5]:
# On charge les 4 fichiers qui nous seront utiles dans 4 dataframe
try:
    data_arbo = read_rome_from_s3( 'unix_arborescence_competences_v459_utf8.csv')
    print(f"Arborescence: Nb de lignes: {data_arbo.shape[0]}, Nb de colonnes: {data_arbo.shape[1]}")
except:
    print('Erreur lors de la lecture du fichier!')

try:
    data_cohe = read_rome_from_s3( 'unix_coherence_item_v459_utf8.csv')
    print(f"Cohérence: Nb de lignes: {data_cohe.shape[0]}, Nb de colonnes: {data_cohe.shape[1]}")
except:
    print('Erreur lors de la lecture du fichier!')

try:
    data_ref_rome = read_rome_from_s3( 'unix_referentiel_code_rome_v459_utf8.csv')
    print(f"Référentiel code ROME: Nb de lignes: {data_ref_rome.shape[0]}, Nb de colonnes: {data_ref_rome.shape[1]}")
except:
    print('Erreur lors de la lecture du fichier!')

try:
    data_ref_comp = read_rome_from_s3( 'unix_referentiel_competence_v459_utf8.csv')
    print(f"Référentiel compétence: Nb de lignes: {data_ref_comp.shape[0]}, Nb de colonnes: {data_ref_comp.shape[1]}")
except:
    print('Erreur lors de la lecture du fichier!')

Arborescence: Nb de lignes: 17758, Nb de colonnes: 11
Cohérence: Nb de lignes: 106224, Nb de colonnes: 3
Référentiel code ROME: Nb de lignes: 1584, Nb de colonnes: 10
Référentiel compétence: Nb de lignes: 18245, Nb de colonnes: 12


In [6]:
# Suppression des valeur NULL:
data_arbo = data_arbo.dropna()

# Modification du type 'code_ogr_competence':
data_arbo['code_ogr_competence'] = data_arbo['code_ogr_competence'].astype('int64')

# Merge des 4 dataframe

In [7]:
# unix_arborescence_competences_v459_utf8.csv
#data_arbo.code_ogr_competence
# unix_coherence_item_v459_utf8.csv
#data_cohe.code_ogr 

df_merge = pd.merge(data_cohe, data_arbo, left_on='code_ogr', right_on='code_ogr_competence').drop(columns=['code_ogr'])
df_merge.shape

(38961, 13)

In [8]:
df_merge.head(20)

,code_rome,coeur_metier,code_domaine_competence,domaine_competence,code_enjeu,enjeu,code_objectif,objectif,code_macro_competence,code_ogr_macro_competence,libelle_macro_competence,code_ogr_competence,libelle_competence
0,A1101,NaN,3,"Production, Construction, Qualité, Logistique",3F,Prévention des risques,3F1,Contrôler la production,3F101,300172,"Contrôler le fonctionnement d'un équipement, d...",104016,Contrôler le fonctionnement d'un dispositif de...
1,A1101,NaN,3,"Production, Construction, Qualité, Logistique",3E,"Maintenance, Réparation",3E1,"Entretenir, corriger",3E103,300181,"Entretenir un équipement, une machine, une ins...",107141,Entretenir les mécanismes d'un engin motorisé
2,A1101,NaN,3,"Production, Construction, Qualité, Logistique",3J,Transport,3J1,Assurer un transport,3J102,300270,Conduire un véhicule routier,110024,Conduire un poids lourd
3,A1101,NaN,3,"Production, Construction, Qualité, Logistique",3H,Gestion des stocks,3H2,"Manutentionner, conditionner des produits",3H201,300262,"Trier, ranger des produits, des matières premi...",117548,Stocker un produit
4,A1101,NaN,3,"Production, Construction, Qualité, Logistique",3C,Aménagement,3C1,Aménager un espace,3C104,300297,"Aménager, entretenir un espace naturel, une forêt",122567,Débroussailler une plantation et ses chemins d...
5,A1101,NaN,3,"Production, Construction, Qualité, Logistique",3C,Aménagement,3C1,Aménager un espace,3C104,300297,"Aménager, entretenir un espace naturel, une forêt",122568,Contrôler l'état d'une plantation
6,A1101,Principale,3,"Production, Construction, Qualité, Logistique",3A,Conception,3A1,"Concevoir, spécifier",3A110,300174,Elaborer et planifier un programme de producti...,122573,"Planifier une opération de semis, de traitemen..."
7,A1101,Principale,3,"Production, Construction, Qualité, Logistique",3B,"Production, Fabrication",3B4,"Cultiver, produire",3B401,300294,Cultiver et récolter des végétaux / minéraux,122730,"Préparer les sols, les plantations (épandage, ..."
8,A1101,NaN,3,"Production, Construction, Qualité, Logistique",3C,Aménagement,3C1,Aménager un espace,3C104,300297,"Aménager, entretenir un espace naturel, une forêt",122817,Réaliser des travaux d'aménagement
9,A1101,Principale,3,"Production, Construction, Qualité, Logistique",3B,"Production, Fabrication",3B4,"Cultiver, produire",3B401,300294,Cultiver et récolter des végétaux / minéraux,122853,Récolter un produit à maturité et selon les co...


In [9]:
# unix_referentiel_code_rome_v459_utf8.csv
#data_ref_rome.code_rome

df_merge = pd.merge(df_merge, data_ref_rome, on='code_rome', suffixes=('_coherence', '_reference'))
df_merge.shape

(38961, 22)

In [10]:
df_merge.head(20)

,code_rome,coeur_metier,code_domaine_competence,domaine_competence,code_enjeu,enjeu,code_objectif,objectif,code_macro_competence,code_ogr_macro_competence,libelle_macro_competence,code_ogr_competence,libelle_competence,code_fiche_metier,code_ogr,libelle_rome,transition_eco,transition_num,transition_demo,emploi_reglemente,emploi_cadre,code_rome_parent
0,A1101,NaN,3,"Production, Construction, Qualité, Logistique",3F,Prévention des risques,3F1,Contrôler la production,3F101,300172,"Contrôler le fonctionnement d'un équipement, d...",104016,Contrôler le fonctionnement d'un dispositif de...,11,6,Conducteur / Conductrice d'engins agricoles,Emploi Blanc,NaN,NaN,NaN,NaN,A1101
1,A1101,NaN,3,"Production, Construction, Qualité, Logistique",3E,"Maintenance, Réparation",3E1,"Entretenir, corriger",3E103,300181,"Entretenir un équipement, une machine, une ins...",107141,Entretenir les mécanismes d'un engin motorisé,11,6,Conducteur / Conductrice d'engins agricoles,Emploi Blanc,NaN,NaN,NaN,NaN,A1101
2,A1101,NaN,3,"Production, Construction, Qualité, Logistique",3J,Transport,3J1,Assurer un transport,3J102,300270,Conduire un véhicule routier,110024,Conduire un poids lourd,11,6,Conducteur / Conductrice d'engins agricoles,Emploi Blanc,NaN,NaN,NaN,NaN,A1101
3,A1101,NaN,3,"Production, Construction, Qualité, Logistique",3H,Gestion des stocks,3H2,"Manutentionner, conditionner des produits",3H201,300262,"Trier, ranger des produits, des matières premi...",117548,Stocker un produit,11,6,Conducteur / Conductrice d'engins agricoles,Emploi Blanc,NaN,NaN,NaN,NaN,A1101
4,A1101,NaN,3,"Production, Construction, Qualité, Logistique",3C,Aménagement,3C1,Aménager un espace,3C104,300297,"Aménager, entretenir un espace naturel, une forêt",122567,Débroussailler une plantation et ses chemins d...,11,6,Conducteur / Conductrice d'engins agricoles,Emploi Blanc,NaN,NaN,NaN,NaN,A1101
5,A1101,NaN,3,"Production, Construction, Qualité, Logistique",3C,Aménagement,3C1,Aménager un espace,3C104,300297,"Aménager, entretenir un espace naturel, une forêt",122568,Contrôler l'état d'une plantation,11,6,Conducteur / Conductrice d'engins agricoles,Emploi Blanc,NaN,NaN,NaN,NaN,A1101
6,A1101,Principale,3,"Production, Construction, Qualité, Logistique",3A,Conception,3A1,"Concevoir, spécifier",3A110,300174,Elaborer et planifier un programme de producti...,122573,"Planifier une opération de semis, de traitemen...",11,6,Conducteur / Conductrice d'engins agricoles,Emploi Blanc,NaN,NaN,NaN,NaN,A1101
7,A1101,Principale,3,"Production, Construction, Qualité, Logistique",3B,"Production, Fabrication",3B4,"Cultiver, produire",3B401,300294,Cultiver et récolter des végétaux / minéraux,122730,"Préparer les sols, les plantations (épandage, ...",11,6,Conducteur / Conductrice d'engins agricoles,Emploi Blanc,NaN,NaN,NaN,NaN,A1101
8,A1101,NaN,3,"Production, Construction, Qualité, Logistique",3C,Aménagement,3C1,Aménager un espace,3C104,300297,"Aménager, entretenir un espace naturel, une forêt",122817,Réaliser des travaux d'aménagement,11,6,Conducteur / Conductrice d'engins agricoles,Emploi Blanc,NaN,NaN,NaN,NaN,A1101
9,A1101,Principale,3,"Production, Construction, Qualité, Logistique",3B,"Production, Fabrication",3B4,"Cultiver, produire",3B401,300294,Cultiver et récolter des végétaux / minéraux,122853,Récolter un produit à maturité et selon les co...,11,6,Conducteur / Conductrice d'engins agricoles,Emploi Blanc,NaN,NaN,NaN,NaN,A1101


In [11]:
# Merge des compétencess
df_merge = pd.merge(
    df_merge,
    data_ref_comp,
    how='inner',  # 'inner' pour ne garder que les correspondances, 'outer' pour tout inclure , left ou right
    left_on=['libelle_competence', 'code_ogr_competence'],
    right_on=['libelle_competence', 'code_ogr'], #liste des colonnes sur lesquelles faire la jointure.
)

df_merge.shape

(38961, 33)

In [15]:
file_name = '/Users/stephane/Documents/Formation - Data 0325/Projet fil rouge/Data/Code ROME/df_competence_rome_complet.csv'
df_merge.to_csv(file_name)

In [13]:
df_merge.head(20)

,code_rome,coeur_metier,code_domaine_competence,domaine_competence,code_enjeu,enjeu,code_objectif,objectif,code_macro_competence,code_ogr_macro_competence,libelle_macro_competence,code_ogr_competence,libelle_competence,code_fiche_metier,code_ogr_x,libelle_rome,transition_eco_x,transition_num_x,transition_demo,emploi_reglemente,emploi_cadre,code_rome_parent,code_ogr_y,nature,definition,code_ogr_macro_comp,code_macro_comp,cat_comp,sous_cat_comp,transition_eco_y,transition_num_y,libelle_heterogeneite,libelle_origine
0,A1101,NaN,3,"Production, Construction, Qualité, Logistique",3F,Prévention des risques,3F1,Contrôler la production,3F101,300172,"Contrôler le fonctionnement d'un équipement, d...",104016,Contrôler le fonctionnement d'un dispositif de...,11,6,Conducteur / Conductrice d'engins agricoles,Emploi Blanc,NaN,NaN,NaN,NaN,A1101,104016,Compétence,NaN,300172.0,NaN,Savoir-faire,Technique expert,NaN,N,NaN,NaN
1,A1101,NaN,3,"Production, Construction, Qualité, Logistique",3E,"Maintenance, Réparation",3E1,"Entretenir, corriger",3E103,300181,"Entretenir un équipement, une machine, une ins...",107141,Entretenir les mécanismes d'un engin motorisé,11,6,Conducteur / Conductrice d'engins agricoles,Emploi Blanc,NaN,NaN,NaN,NaN,A1101,107141,Compétence,NaN,300181.0,NaN,Savoir-faire,Technique,NaN,N,NaN,NaN
2,A1101,NaN,3,"Production, Construction, Qualité, Logistique",3J,Transport,3J1,Assurer un transport,3J102,300270,Conduire un véhicule routier,110024,Conduire un poids lourd,11,6,Conducteur / Conductrice d'engins agricoles,Emploi Blanc,NaN,NaN,NaN,NaN,A1101,110024,Compétence,NaN,300270.0,NaN,Savoir-faire,Technique,NaN,N,NaN,NaN
3,A1101,NaN,3,"Production, Construction, Qualité, Logistique",3H,Gestion des stocks,3H2,"Manutentionner, conditionner des produits",3H201,300262,"Trier, ranger des produits, des matières premi...",117548,Stocker un produit,11,6,Conducteur / Conductrice d'engins agricoles,Emploi Blanc,NaN,NaN,NaN,NaN,A1101,117548,Compétence,NaN,300262.0,NaN,Savoir-faire,Technique,NaN,N,NaN,NaN
4,A1101,NaN,3,"Production, Construction, Qualité, Logistique",3C,Aménagement,3C1,Aménager un espace,3C104,300297,"Aménager, entretenir un espace naturel, une forêt",122567,Débroussailler une plantation et ses chemins d...,11,6,Conducteur / Conductrice d'engins agricoles,Emploi Blanc,NaN,NaN,NaN,NaN,A1101,122567,Compétence,NaN,300297.0,NaN,Savoir-faire,Technique,O,N,NaN,NaN
5,A1101,NaN,3,"Production, Construction, Qualité, Logistique",3C,Aménagement,3C1,Aménager un espace,3C104,300297,"Aménager, entretenir un espace naturel, une forêt",122568,Contrôler l'état d'une plantation,11,6,Conducteur / Conductrice d'engins agricoles,Emploi Blanc,NaN,NaN,NaN,NaN,A1101,122568,Compétence,NaN,300297.0,NaN,Savoir-faire,Technique,NaN,N,NaN,NaN
6,A1101,Principale,3,"Production, Construction, Qualité, Logistique",3A,Conception,3A1,"Concevoir, spécifier",3A110,300174,Elaborer et planifier un programme de producti...,122573,"Planifier une opération de semis, de traitemen...",11,6,Conducteur / Conductrice d'engins agricoles,Emploi Blanc,NaN,NaN,NaN,NaN,A1101,122573,Compétence,NaN,300174.0,NaN,Savoir-faire,Technique expert,NaN,N,NaN,NaN
7,A1101,Principale,3,"Production, Construction, Qualité, Logistique",3B,"Production, Fabrication",3B4,"Cultiver, produire",3B401,300294,Cultiver et récolter des végétaux / minéraux,122730,"Préparer les sols, les plantations (épandage, ...",11,6,Conducteur / Conductrice d'engins agricoles,Emploi Blanc,NaN,NaN,NaN,NaN,A1101,122730,Compétence,NaN,300294.0,NaN,Savoir-faire,Technique,NaN,N,NaN,NaN
8,A1101,NaN,3,"Production, Construction, Qualité, Logistique",3C,Aménagement,3C1,Aménager un espace,3C104,300297,"Aménager, entretenir un espace naturel, une forêt",122817,Réaliser des travaux d'aménagement,11,6,Conducteur / Conductrice d'engins agricoles,Emploi Blanc,NaN,NaN,NaN,NaN,A1101,122817,Compétence,NaN,300297.0,NaN,Savoir-faire,Technique,NaN,N,NaN,NaN
9,A1101,Principale,3,"Production, Construction, Qualité, Logistique",3B,"Production, Fab

In [14]:
df_merge[[
    'code_rome', 
	'libelle_rome', 
	'emploi_cadre',
    'code_ogr_competence', 
	'libelle_competence',
    'cat_comp', 
	'sous_cat_comp',
    'enjeu',
    'objectif',
    'libelle_macro_competence', 
	'domaine_competence',
    'coeur_metier', 
	'transition_eco_x', 
	'transition_num_x',
	'transition_eco_y', 
	'transition_num_y'
]]

,code_rome,libelle_rome,emploi_cadre,code_ogr_competence,libelle_competence,cat_comp,sous_cat_comp,enjeu,objectif,libelle_macro_competence,domaine_competence,coeur_metier,transition_eco_x,transition_num_x,transition_eco_y,transition_num_y
0,A1101,Conducteur / Conductrice d'engins agricoles,NaN,104016,Contrôler le fonctionnement d'un dispositif de...,Savoir-faire,Technique expert,Prévention des risques,Contrôler la production,"Contrôler le fonctionnement d'un équipement, d...","Production, Construction, Qualité, Logistique",NaN,Emploi Blanc,NaN,NaN,N
1,A1101,Conducteur / Conductrice d'engins agricoles,NaN,107141,Entretenir les mécanismes d'un engin motorisé,Savoir-faire,Technique,"Maintenance, Réparation","Entretenir, corriger","Entretenir un équipement, une machine, une ins...","Production, Construction, Qualité, Logistique",NaN,Emploi Blanc,NaN,NaN,N
2,A1101,Conducteur / Conductrice d'engins agricoles,NaN,110024,Conduire un poids lourd,Savoir-faire,Technique,Transport,Assurer un transport,Conduire un véhicule routier,"Production, Construction, Qualité, Logistique",NaN,Emploi Blanc,NaN,NaN,N
3,A1101,Conducteur / Conductrice d'engins agricoles,NaN,117548,Stocker un produit,Savoir-faire,Technique,Gestion des stocks,"Manutentionner, conditionner des produits","Trier, ranger des produits, des matières premi...","Production, Construction, Qualité, Logistique",NaN,Emploi Blanc,NaN,NaN,N
4,A1101,Conducteur / Conductrice d'engins agricoles,NaN,122567,Débroussailler une plantation et ses chemins d...,Savoir-faire,Technique,Aménagement,Aménager un espace,"Aménager, entretenir un espace naturel, une forêt","Production, Construction, Qualité, Logistique",NaN,Emploi Blanc,NaN,O,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38956,N4403,Opérateur / Opératrice ferroviaire,N,507903,Evaluer les risques liés aux différentes opéra...,Savoir-faire,Technique expert,Prévention des risques,"Analyser, prévenir les risques","Evaluer, prévenir, et gérer les risques et la ...","Production, Construction, Qualité, Logistique",Principale,Emploi stratégique pour la Transition écologique,N,N,N
38957,N4403,Opérateur / Opératrice ferroviaire,N,512818,Appliquer les protocoles de sécurité ferroviaires,Savoir-faire,Technique expert,Prévention des risques,"Analyser, prévenir les risques","Evaluer, prévenir, et gérer les risques et la ...","Production, Construction, Qualité, Logistique",Principale,Emploi stratégique pour la Transition écologique,N,NaN,NaN
38958,N4403,Opérateur / Opératrice ferroviaire,N,512855,Atteler/dételer les wagons d'un train,Savoir-faire,Technique,"Production, Fabrication","Mettre en oeuvre des procédés, modes opératoires",Mettre en oeuvre les processus et les modes op...,"Production, Construction, Qualité, Logistique",Principale,Emploi stratégique pour la Transition écologique,N,NaN,NaN
38959,N4403,Opérateur / Opératrice ferroviaire,N,513079,"Identifier des dangers, problèmes ou besoins m...",Savoir-faire,Technique expert,Prévention des risques,"Analyser, prévenir les risques","Evaluer, prévenir, et gérer les risques et la ...","Production, Construction, Qualité, Logistique",NaN,Emploi stratégique pour la Transition écologique,N,NaN,NaN


In [17]:
file_name = '/Users/stephane/Documents/Formation - Data 0325/Projet fil rouge/Data/Code ROME/df_competence_rome_eda_v2.csv'
df_merge[[
    'code_rome', 
	'libelle_rome', 
	'emploi_cadre',
    'code_ogr_competence', 
	'libelle_competence',
    'cat_comp', 
	'sous_cat_comp',
    'enjeu',
    'objectif',
    'libelle_macro_competence', 
	'domaine_competence',
    'coeur_metier', 
	'transition_eco_x', 
	'transition_num_x',
	'transition_eco_y', 
	'transition_num_y'
]].to_csv(file_name)